# QMUL Careers Team Python Script
### A detailed guide on how to run this programme.



Each text box describe what the code does in the cell below it.

The first cell imports some of the software that the programme needs, if you click the small play button above, or press Shift and Enter, it will run the cell.

In [114]:
import pandas as pd
import openpyxl
import numpy as np
import os
import ipywidgets as widgets

We need the excel workbook that the programme will use, and within that the excel spreadsheet with the student information. Please name the spreadsheet with the application information: "data" and save it on your computer.

The video below shows how to add this to the programme.

In [ ]:
from IPython.display import Video; from ipywidgets import interactive, IntSlider

vid = Video(filename="media/add_applications.mov",data="", width=800, height = 600)
display(vid)



Next, point the programme at the excel sheet just uploaded.

In [115]:
def get_sheet():
    for file in os.listdir('.'):
        if file.endswith(".xlsx"):
            return file

filename = get_sheet()

Read the spreadsheet in, and perform some data cleaning.

In [116]:
df = pd.read_excel(filename,"data")


Any missing scores will be replaced with the average score of the dataset.

In [117]:
df['Total score'].fillna(df['Total score'].mean(), inplace=True)

Some exploratory data analysis in the next 2 cells

In [118]:
df['Gender'].value_counts()

F    42
M    18
Name: Gender, dtype: int64

In [119]:
df['YearOfStudy'].value_counts()

2    32
3    20
1     5
4     3
Name: YearOfStudy, dtype: int64

Take only the columns important to establishing the groups. 

In [120]:
group_df = df[[
'email', 
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Total score',
'Personality type']]

'email', 
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Total score',
'Personality type'

In [121]:
group_df = group_df.assign(groupno='')


You can decide which variables you would like to prioritize your sort by.

In [122]:
the_vars = widgets.SelectMultiple(
    options=[
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Personality type'],
    value=['Gender'],
    #rows=10,
    description='Scores',
    disabled=False
)
the_vars

SelectMultiple(description='Scores', index=(2,), options=('Course', 'YearOfStudy', 'Gender', 'Bursary holder',…

The script will always prioritise total score first and foremost.

In [124]:
to_sort = list(the_vars.value)
to_sort.insert(0,'Total score')
to_sort

['Total score', 'Gender']


#### The Sorting Algorithm

In [125]:
def assign_group(df):
    df.sort_values(by=to_sort, ascending=False, inplace=True)
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/6,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/6,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

assigned = assign_group(group_df)

    

More data analysis for the different groups

In [133]:
assigned.head()

,email,Course,YearOfStudy,Gender,Bursary holder,Client meetings delivery,Total score,Personality type,groupno
25,Laoise.O'Dwyer@gmail.com,BA FT English,2,F,No,Face to face / online,12.0,Advocate INFJ,1
44,Nelson.Churro@gmail.com,LLB FT Law,3,M,No,Face to face / online,11.5,Commander ENTJ,2
12,Barry.McCullagh@gmail.com,BA FT Politics with Business Management,2,F,Yes,Face to face / online,11.5,Protagonist ENFJ,3
23,Raxit.Paul Louis@gmail.com,BSc FT Biomedical Sciences,2,F,Yes,Face to face / online,11.5,Consul ESFJ,4
37,Andre.Campana@gmail.com,BSc FT Mathematics with Finance and Accounting,3,M,No,Face to face / online,11.0,Consul ESFJ,5


In [132]:
assigned.groupby(['groupno'])[['Total score']].mean()

,Total score
groupno,
1,7.000000
2,7.083333
3,7.250000
4,7.416667
5,7.333333
6,7.250000
7,7.250000
8,7.083333
9,7.028736


In [ ]:
assigned.pivot_table(index=['groupno'], columns=['YearOfStudy'], aggfunc='size', fill_value=0)


In [ ]:
assigned.pivot_table(index=['groupno'], columns=['Gender'], aggfunc='size', fill_value=0)


### Send the groups back into the original spreadsheet

In [ ]:
subset = assigned[['email','groupno']]

final = pd.merge(df,subset, how='inner', on='email')



In [ ]:
book = openpyxl.load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book
final.to_excel(writer, sheet_name='New Sheet Groups')
writer.save()
writer.close()

## Make sure you download your new spreadsheet, you can copy and paste it into your previous working document.